In [1]:
# load data
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.io as sciio
from time import time
import sklearn
# %matplotlib inline
from sklearn.datasets import make_blobs
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import skimage
from sklearn import svm, metrics, datasets
from sklearn.svm import SVC
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.transform import resize as imresize

REAL_IMAG = False
NO_NMF = True


# Load images in structured directory like it's sklearn sample dataset
def load_image_files(container_path, dimension=(64, 64)):  # 调整图片的尺寸为dimension=(64, 64)

    image_dir = Path(container_path)
    # folders is the list of folders each conains a category of data
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    # _______________________________________________________
    # 去掉annotations文件夹
    folders = folders[:-1]
    # _______________________________________________________
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []

    for i, direc in enumerate(folders):
        # print(i)
        # print(direc)
        for file in direc.iterdir():
            mat_data = sciio.loadmat(file)
            img = np.abs(mat_data['frame_Ev'])
            raw_data = []
            if REAL_IMAG:
                raw_data.extend(np.real(mat_data['frame_Ev']).flatten())
                raw_data.extend(np.imag(mat_data['frame_Ev']).flatten())
                raw_data.extend(np.real(mat_data['frame_Eh']).flatten())
                raw_data.extend(np.imag(mat_data['frame_Eh']).flatten())
                raw_data = np.abs(raw_data)
            else:
                raw_data.extend(np.abs(mat_data['frame_Ev']).flatten())
                raw_data.extend(np.abs(mat_data['frame_Eh']).flatten())
            flat_data.append(raw_data)
            images.append(img)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)
    print("fucked")

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)


# ----KC501-----
# image_dataset = load_image_files('/home/kc501/LJY/Alexnet/dataRCS')

# ----ICraft----
t_load_data = time()
image_dataset = load_image_files('.\dataRCS')
print("Data load done in %0.3fs" % (time() - t_load_data))
# image_dataset_test = load_image_files("E:/RL_code/alex-net-image-classification-master/class3/val")
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.2, random_state=101)


ValueError: Unknown mat file type, version 100, 97

In [ ]:
n_components=200
if not NO_NMF:
    t_NMF = time()
    nmf = NMF(n_components=n_components, init='nndsvd', tol=1e-2, max_iter=1000, verbose=True).fit(X_train)
    # W = nmf.components_.reshape((n_components, 64, 64))

    X_train_nmf = nmf.transform(X_train)
    X_test_nmf = nmf.transform(X_test)
    print("NMF done in %0.3fs" % (time() - t_NMF))

In [ ]:
f = open('./TZB.txt', 'w')
t_train = time()
clf = SVC(C=1, kernel='poly', gamma=0.01, class_weight='balanced', decision_function_shape='ovo', verbose=True)
print("SVM done in %0.3fs" % (time() - t_train))

if NO_NMF:
    clf = clf.fit(X_train, y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
else:
    clf = clf.fit(X_train_nmf, y_train)
    y_pred_train = clf.predict(X_train_nmf)
    y_pred_test = clf.predict(X_test_nmf)

y_true_train = y_train
y_true_test = y_test

f.write("n_components: {}\n".format(n_components))
f.write("Acc on train data: {}\n".format(accuracy_score(y_true_train, y_pred_train)))
f.write("Acc on test data: {}\n".format(accuracy_score(y_true_test, y_pred_test)))
f.write("------------------------------------------------------------------------------\n")
print("n_components: ", n_components)
print("Acc on train data:" + str(accuracy_score(y_true_train, y_pred_train)))

print("Acc on test data:" + str(accuracy_score(y_true_test, y_pred_test)))